# 目标：use a multi-layer perceptron model to model the data in the “classification”

In [3]:

import pandas as pd
import numpy as np
import os 

data_dir = '/media/data1/hy/OrangeHomeWork'


train_data_path = os.path.join(data_dir, 'SLTA_Projects/dataset/classification/classification_train.csv')
eval_data_path = os.path.join(data_dir,  'SLTA_Projects/dataset/classification/classification_val.csv')
test_data_path = os.path.join(data_dir, 'SLTA_Projects/dataset/classification/classification_test.csv')

output_csv_path = os.path.join(data_dir, 'mlp_classification_test.csv')

train_data_df = pd.read_csv(train_data_path)
eval_data_df = pd.read_csv(eval_data_path)
test_data_df = pd.read_csv(test_data_path)

# 数据EDA

In [4]:
train_data_df.info(),  eval_data_df.info(), test_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   x_1     10000 non-null  float64
 1   x_2     10000 non-null  float64
 2   y       10000 non-null  int64  
dtypes: float64(2), int64(1)
memory usage: 234.5 KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   x_1     1000 non-null   float64
 1   x_2     1000 non-null   float64
 2   y       1000 non-null   int64  
dtypes: float64(2), int64(1)
memory usage: 23.6 KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   x_1     1000 non-null   float64
 1   x_2     1000 non-null   float64
 2   y       0 non-null      float64
dtypes: float64(3)
memory

(None, None, None)

In [5]:
train_data_df.shape, eval_data_df.shape, test_data_df.shape

((10000, 3), (1000, 3), (1000, 3))

In [6]:
train_data_df.head(5)

,x_1,x_2,y
0,-1.273580,0.642793,1
1,-0.099602,0.840770,1
2,-1.010065,-1.857753,0
3,0.895425,-0.831040,0
4,1.043244,-1.045208,0


In [7]:
eval_data_df.head(5)

,x_1,x_2,y
0,0.956742,-2.565025,0
1,1.085391,1.019169,1
2,-1.245391,-2.297339,0
3,0.576724,-1.675460,0
4,2.745909,1.017409,1


In [8]:
test_data_df.head(5)

,x_1,x_2,y
0,0.646227,0.801867,NaN
1,-0.070090,1.745161,NaN
2,-0.797350,-0.790427,NaN
3,0.579818,-2.036111,NaN
4,-1.253754,-0.351167,NaN


# 训练集、验证集、测试集划分

In [10]:
# 训练数据集
X_train, y_train = train_data_df.iloc[:, :-1], train_data_df.iloc[:, 2]
# 验证集
X_eval, y_eval = eval_data_df.iloc[:, :-1], eval_data_df.iloc[:, 2]
#测试集
X_test = test_data_df.iloc[:, : - 1]

# 使用numpy实现多层感知机

In [18]:
import numpy as np

# 激活函数及其导数
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    return x * (1 - x)

# 损失函数（均方误差）
def mse_loss(y_true, y_pred):
    return np.mean((y_true - y_pred) ** 2)

# 多层感知器类
class MLP:
    def __init__(self, input_size, hidden_size, output_size, learning_rate=0.01):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.learning_rate = learning_rate
        
        # 初始化权重和偏置
        self.weights_input_hidden = np.random.randn(input_size, hidden_size)
        self.bias_hidden = np.zeros(hidden_size)
        self.weights_hidden_output = np.random.randn(hidden_size, output_size)
        self.bias_output = np.zeros(output_size)
    
    def forward(self, x):
        # 前向传播
        self.hidden = sigmoid(np.dot(x, self.weights_input_hidden) + self.bias_hidden)
        self.output = sigmoid(np.dot(self.hidden, self.weights_hidden_output) + self.bias_output)
        return self.output
    
    def backward(self, x, y, output):
        # 反向传播
        output_error = y - output
        output_delta = output_error * sigmoid_derivative(output)
        print("output_delta:", output_delta.shape)
        hidden_error = output_delta.dot(self.weights_hidden_output.T)
        hidden_delta = hidden_error * sigmoid_derivative(self.hidden)
        
        # 更新权重和偏置
        self.weights_hidden_output += self.hidden.T.dot(output_delta) * self.learning_rate
        self.bias_output += np.sum(output_delta, axis=0) * self.learning_rate
        self.weights_input_hidden += x.T.dot(hidden_delta) * self.learning_rate
        self.bias_hidden += np.sum(hidden_delta, axis=0) * self.learning_rate
    
    def train(self, x_train, y_train, epochs=1000):
        for epoch in range(epochs):
            output = self.forward(x_train)
            # print("x_train:", x_train)
            # print("y_train:", y_train)
            # print("output:", output)
            self.backward(x_train, y_train, output)
            if epoch % 20 == 0:
                loss = mse_loss(y_train, output)
                print(f'Epoch {epoch}, Loss: {loss}')
    
    def predict(self, x):
        return self.forward(x)


In [22]:
y_train.to_numpy()

array([1, 1, 0, ..., 1, 1, 0])

In [23]:
from sklearn.metrics import accuracy_score

# 初始化并训练模型
mlp = MLP(input_size=2, hidden_size=2, output_size=1, learning_rate=0.1)

# 训练模型
mlp.train(X_train.to_numpy(), y_train.to_numpy(), epochs=1)





x_train: [[-1.27358015  0.64279291]
 [-0.09960213  0.84076983]
 [-1.01006545 -1.85775309]
 ...
 [ 2.41283545  1.24092761]
 [ 2.21123988  1.13460519]
 [ 0.83575821 -1.47963722]]
y_train: [1 1 0 ... 1 1 0]
output: [[0.23979351]
 [0.27010978]
 [0.1609333 ]
 ...
 [0.31775743]
 [0.31182623]
 [0.18760141]]


ValueError: shapes (10000,10000) and (1,2) not aligned: 10000 (dim 1) != 1 (dim 0)

In [ ]:
# 在验证集上预测
eval_pred = mlp.predict(X_eval)

In [ ]:
# 训练模型
ada.fit(X_train, y_train)

# 在验证集上预测
eval_pred = ada.predict(X_eval)

# 在验证集上计算准确率
accuracy = accuracy_score(y_eval, eval_pred)
print(f"Accuracy: {accuracy:.2f}")